<a href="https://colab.research.google.com/github/Takrimelahichowdhury/Weight-Gain-Prediction/blob/master/WeightGainPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
exercise_df1 = pd.read_csv(io.StringIO(uploaded['exercise_dataset.csv'].decode('utf-8')))

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
exercise_df2 = pd.read_csv(io.StringIO(uploaded['exercise_dataset_2.csv'].decode('utf-8')))

In [ ]:
print(exercise_df1.head())
print(exercise_df2.head())


In [ ]:
print(exercise_df2.isnull().sum())


ID                    0
Exercise              0
Calories Burn         0
Dream Weight          0
Actual Weight         0
Age                   0
Gender                0
Duration              0
Heart Rate            0
BMI                   0
Weather Conditions    0
Exercise Intensity    0
dtype: int64


In [ ]:
exercise_df2 = pd.get_dummies(exercise_df2, columns=['Gender', 'Weather Conditions'], drop_first=True)


In [ ]:
exercise_df2['Weight Difference'] = exercise_df2['Dream Weight'] - exercise_df2['Actual Weight']


In [ ]:
print(exercise_df2.head())


   ID     Exercise  Calories Burn  Dream Weight  Actual Weight  Age  Duration  \
0   1   Exercise 2     286.959851     91.892531      96.301115   45        37   
1   2   Exercise 7     343.453036     64.165097      61.104668   25        43   
2   3   Exercise 4     261.223465     70.846224      71.766724   20        20   
3   4   Exercise 5     127.183858     79.477008      82.984456   33        39   
4   5  Exercise 10     416.318374     89.960226      85.643174   29        34   

   Heart Rate        BMI  Exercise Intensity  Gender_Male  \
0         170  29.426275                   5            1   
1         142  21.286346                   5            1   
2         148  27.899592                   4            1   
3         170  33.729552                  10            1   
4         118  23.286113                   3            0   

   Weather Conditions_Rainy  Weather Conditions_Sunny  Weight Difference  
0                         1                         0          -4.40858

In [ ]:
from sklearn.model_selection import train_test_split

# Defining our features and target
X = exercise_df2[['Actual Weight', 'Age', 'Gender_Male', 'Dream Weight', 'Duration', 'Heart Rate', 'BMI', 'Exercise Intensity', 'Weather Conditions_Rainy', 'Weather Conditions_Sunny']]
y = exercise_df2['Calories Burn']

# Splitting the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Initialize the model
lr_model = LinearRegression()

# Train the model
lr_model.fit(X_train, y_train)

# Predict on the training set
train_preds = lr_model.predict(X_train)

# Calculate the mean squared error on the training data
mse_train = mean_squared_error(y_train, train_preds)
print(f"Training Mean Squared Error: {mse_train}")


Training Mean Squared Error: 13435.92406084098


In [ ]:
# Predict on the testing set
test_preds = lr_model.predict(X_test)

# Calculate the mean squared error on the testing data
mse_test = mean_squared_error(y_test, test_preds)
print(f"Testing Mean Squared Error: {mse_test}")


Testing Mean Squared Error: 13223.399280790924


In [ ]:
def recommend_exercise_and_calories(weight, age, gender_male, dream_weight, duration, heart_rate, bmi, intensity, rainy, sunny):
    # Predicting the calories to burn
    input_data = [[weight, age, gender_male, dream_weight, duration, heart_rate, bmi, intensity, rainy, sunny]]
    predicted_calories = lr_model.predict(input_data)[0]

    # Find the closest matching exercise
    exercise_df1['calorie_difference'] = abs(exercise_df1['Calories per kg'] * weight - predicted_calories)
    recommended_exercise = exercise_df1.loc[exercise_df1['calorie_difference'].idxmin()]['Activity, Exercise or Sport (1 hour)']

    return recommended_exercise, predicted_calories

In [ ]:
user_weight = 90  # for example
user_age = 75
user_gender = 0  # Male: 1, Female: 0
user_dream_weight = 75
user_duration = 30
user_heart_rate = 160
user_bmi = 28
user_intensity = 8
user_rainy = 1
user_sunny = 0

# Call the function
exercise, calories = recommend_exercise_and_calories(user_weight, user_age, user_gender, user_dream_weight, user_duration, user_heart_rate, user_bmi, user_intensity, user_rainy, user_sunny)

print(f"Recommended Exercise: {exercise}")
print(f"Predicted Calories to Burn: {calories}")


In [ ]:
!pip install tensorflow


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Build a simple dense model with input shape as the number of features in X_train
# Modified model for multi-output regression
model = keras.Sequential([
    layers.Input(shape=(X_train.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(2)  # Two output neurons for regression: calories and weight difference
])


model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the model with X_train and y_train
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
78/78 [==============================] - 1s 3ms/step - loss: 55520.7500 - val_loss: 13986.9561
Epoch 2/50
78/78 [==============================] - 0s 2ms/step - loss: 14842.0596 - val_loss: 13869.7178
Epoch 3/50
78/78 [==============================] - 0s 2ms/step - loss: 14769.5859 - val_loss: 13824.1826
Epoch 4/50
78/78 [==============================] - 0s 2ms/step - loss: 14785.8076 - val_loss: 13775.9189
Epoch 5/50
78/78 [==============================] - 0s 2ms/step - loss: 14654.4873 - val_loss: 13742.7129
Epoch 6/50
78/78 [==============================] - 0s 2ms/step - loss: 14653.8154 - val_loss: 13709.9971
Epoch 7/50
78/78 [==============================] - 0s 2ms/step - loss: 14619.4404 - val_loss: 13694.1895
Epoch 8/50
78/78 [==============================] - 0s 2ms/step - loss: 14593.3984 - val_loss: 13722.4834
Epoch 9/50
78/78 [==============================] - 0s 2ms/step - loss: 14554.4160 - val_loss: 13687.7090
Epoch 10/50
78/78 [===========================

In [ ]:
loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {loss}")


25/25 [==============================] - 0s 1ms/step - loss: 14430.4609
Test Loss: 14430.4609375


In [ ]:
# Save the model in TensorFlow SavedModel format
model.save("calories_exercise_model")

# Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("calories_exercise_model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
def recommend_exercise_tf_model(weight, age, gender_male, dream_weight, duration, heart_rate, bmi, intensity, rainy, sunny):
    # Predicting the calories to burn using the TensorFlow model
    input_data = [[weight, age, gender_male, dream_weight, duration, heart_rate, bmi, intensity, rainy, sunny]]
    predicted_values = model.predict(input_data)
    predicted_calories = predicted_values[0][0]  # Assuming calories prediction is the first output

    # Find the closest matching exercise
    exercise_df1['calorie_difference'] = abs(exercise_df1['Calories per kg'] * weight - predicted_calories)
    recommended_exercise = exercise_df1.loc[exercise_df1['calorie_difference'].idxmin()]['Activity, Exercise or Sport (1 hour)']

    return recommended_exercise, predicted_calories



In [ ]:
# Test the function
exercise, calories = recommend_exercise_tf_model(user_weight, user_age, user_gender, user_dream_weight, user_duration, user_heart_rate, user_bmi, user_intensity, user_rainy, user_sunny)

print(f"Recommended Exercise (TF Model): {exercise}")
print(f"Predicted Calories to Burn (TF Model): {calories}")

1/1 [==============================] - 0s 64ms/step
Recommended Exercise (TF Model): Cross country skiing, uphill
Predicted Calories to Burn (TF Model): 278.5006103515625
